In [2]:
code = '''
class Block(ABC):
    def __init__(
        self,
        template: str,
        role: str = "user",
        model_name: str = "gpt-3.5-turbo-16k",
        temperature: float = 0.1,
    ):
        """
        Super simple interface for chat-like GPT completions

        template (str): The template to use for the model
        role (str, optional): The role of the user. Defaults to "user".
        model_name (str, optional): The model name to use. Defaults to "gpt-3.5-turbo".
        temperature (float, optional): The temperature to use. Defaults to 0.2.
        stream (bool, optional): Whether to stream the output or not. Defaults to False.
        """
        self.template = template
        self.input_variables = self.get_input_variables()
        self.message = {"role": role, "content": None}
        self.model_name = model_name
        self.temperature = temperature
        self.logs = []

    def get_input_variables(self):
        return re.findall(r"\{(\w+)\}", self.template)

    def log(self, request, response, response_time=None):
        log_entry = {
            "inputs": request,
            "response": response,
        }
        if response_time:
            log_entry["response_time"] = response_time
        self.logs.append(log_entry)

    def create_completion(
        self, inputs: Dict[str, Any]
    ) -> Union[Dict[str, Any], Generator]:
        """Create a GPT completion"""
        self.message["content"] = self.template.format(**inputs)
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=[self.message],
            temperature=self.temperature,
            stream=self.stream,
        )
        return response

    @abstractmethod
    def execute(self, inputs: str) -> Union[str, Generator]:
        pass

    @abstractmethod
    def display(self, content) -> None:
        pass

    def __call__(self, *args, **kwargs) -> Union[str, Generator]:
        inputs = {}
        if args:
            inputs = {key: value for key, value in zip(self.input_variables, args)}
        if kwargs:
            inputs.update(kwargs)
        return self.execute(inputs)
'''

In [1]:
%reload_ext autoreload
%autoreload 2

from llm_blocks import blocks

In [3]:
template = """
Here is {symbol} from {reponame}:
{code}

Can you convert this python code to pseudocode? Focus on the high-level logic not syntax or naming. Be concise.
"""

In [4]:
block = blocks.StreamBlock(template=template)
response_generator = block(symbol="llm_blocks/blocks.py Block", reponame="llm-blocks", code=code)
block.display(response_generator)

Pseudocode for the given Python code:

1. Define a class named "Block" that inherits from the ABC class.
2. Initialize the Block class with the following parameters: template, role, model_name, and temperature.
3. Set the template, input_variables, message, model_name, temperature, and logs attributes of the Block class.
4. Define a method named "get_input_variables" that returns a list of input variables extracted from the template.
5. Define a method named "log" that takes request, response, and response_time as parameters and appends a log entry to the logs attribute.
6. Define a method named "create_completion" that takes inputs as a dictionary and creates a GPT completion using the OpenAI API.
7. Define abstract methods named "execute" and "display" that need to be implemented by subclasses.
8. Define the "__call__" method that takes arguments as inputs and executes the "execute" method with the provided inputs.
9. If arguments are provided, create a dictionary of inputs using the

In [5]:
block.logs

[{'inputs': {'symbol': 'llm_blocks/blocks.py Block',
   'reponame': 'llm-blocks',
   'code': '\nclass Block(ABC):\n    def __init__(\n        self,\n        template: str,\n        role: str = "user",\n        model_name: str = "gpt-3.5-turbo-16k",\n        temperature: float = 0.1,\n    ):\n        """\n        Super simple interface for chat-like GPT completions\n\n        template (str): The template to use for the model\n        role (str, optional): The role of the user. Defaults to "user".\n        model_name (str, optional): The model name to use. Defaults to "gpt-3.5-turbo".\n        temperature (float, optional): The temperature to use. Defaults to 0.2.\n        stream (bool, optional): Whether to stream the output or not. Defaults to False.\n        """\n        self.template = template\n        self.input_variables = self.get_input_variables()\n        self.message = {"role": role, "content": None}\n        self.model_name = model_name\n        self.temperature = temperature

In [6]:
block = blocks.BatchBlock(template=template, model_name="gpt-3.5-turbo")
response = block(symbol="llm_blocks/blocks.py Block", reponame="llm-blocks", code=code)
block.display(response)

{
  "id": "chatcmpl-7pzNvAtrJiLuQqwlJTCTQ9BrPUtqR",
  "object": "chat.completion",
  "created": 1692625047,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Pseudocode for the given Python code:\n\n1. Define a class called \"Block\" that inherits from the ABC class.\n2. Initialize the Block class with the following parameters: template, role, model_name, and temperature.\n3. Set the template, input_variables, message, model_name, temperature, and logs attributes of the Block class.\n4. Define a method called \"get_input_variables\" that returns a list of input variables extracted from the template.\n5. Define a method called \"log\" that takes request, response, and response_time as parameters and appends a log entry to the logs attribute.\n6. Define a method called \"create_completion\" that takes inputs as a dictionary and creates a GPT completion using the OpenAI API.\n7. Define abstract metho

In [7]:
block.logs[0].keys()

dict_keys(['inputs', 'response'])